In [2]:
%load_ext autoreload
%autoreload 2
from vpot.calc import myMolecule, sphericalGrid, blockGrid
from vpot.calc.potential import vpot,vBpot, vpotANC
from vpot.calc import DFTGroundState
from matplotlib import pyplot as plt
import psi4
import numpy as np
import logging,time
from ase.io import read,write
logging.basicConfig(filename='benchmark.log', level=logging.INFO,filemode="w")

In [13]:
A = read("tests/6-QM7/dsgdb7njpNewline.xyz",index=":")

In [14]:
X = []
for c,i in enumerate(A):
    X.append([c,len(set(i.get_atomic_numbers()))])
S = sorted(X,key=lambda x:x[1])
print(S[-1])
A[S[-1][0]].write("hetero.xyz")

[7203, 5]


In [ ]:
M  = myMolecule("hetero.xyz","def2-QZVP")
Gs = sphericalGrid(M,minDist=0.0,maxDist=10.0,nRadial=250,nSphere=590)
VMat = Gs.optimizeBasis(potentialType="anc",a=3)
Gs.printStats(VMat,output="print")
Gs.exportErrorVsDistance(VMat)

NameError: name 'myMolecule' is not defined

In [16]:
Gs.phi.shape

(612525, 442)

In [ ]:
prec = 2

M  = myMolecule("hetero.xyz","def2-QZVP-decon")
Gs = sphericalGrid(M,minDist=0.0,maxDist=10.0,nRadial=250,nSphere=590,pruningScheme="None")
VMat = Gs.optimizeBasis(potentialType="anc",a=prec)
Gs.printStats(VMat,output="print")
Gs.exportErrorVsDistance(VMat)


VANCbasis = np.einsum("ji,j,jk,j->ik",Gs.phi,vBpot(Gs.phi,VMat.diagonal()),Gs.phi,Gs.weights,optimize=True)
VANCexact = np.einsum("ji,j,jk,j->ik",Gs.phi,vpotANC(M.geom,M.elez,Gs.points,prec),Gs.phi,Gs.weights,optimize=True)
VPOT = M.ao_pot

#basis error; anc error
print(f"{np.linalg.norm(VANCbasis-VANCexact)} {np.linalg.norm(VANCexact-VPOT)}")

   => Loading Basis Set <=

    Name: DEF2-QZVP-DECON
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3 entry O          line   290 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs func decontract
    atoms 2    entry S          line   744 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs func decontract
    atoms 4-6  entry C          line   194 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs func decontract
    atoms 7    entry N          line   242 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs func decontract
    atoms 8    entry H          line    15 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs func decontract



In [28]:
%%time
VB = vBpot(Gs.phi,VMat.diagonal())
I1 = VB*Gs.weights
VANCbasis2 = np.einsum("ji,j,jk->ik",Gs.phi,I1,Gs.phi,optimize=True)


CPU times: user 2.1 s, sys: 80 ms, total: 2.18 s
Wall time: 2.18 s


In [24]:
np.sqrt(-1)

/tmp/ipykernel_10870/3438155168.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(-1)


nan

In [18]:
MyDFTbasis,_ = DFTGroundState(M,"PBE",AOPOT=VANCbasis)

PREFIX               VPOT                
E_CONV               1e-08               
D_CONV               1e-06               
MAXITER              150                 
BASIS                def2-QZVP           
GAMMA                0.95                
DIIS_LEN             6                   
DIIS_MODE            ADIIS+CDIIS         
DIIS_EPS             0.1                 
MIXMODE              DAMP                
RESTART              False               


=Entering Ground State Kohn-Sham=


Options Run:



Basis Set:

   => Loading Basis Set <=

    Name: DEF2-QZVP
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3 entry O          line   290 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs 
    atoms 2    entry S          line   744 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs 
    atoms 4-6  entry C          line   194 file /media/miniconda3/envs/vpot/share/psi4/basis/def2-qzvp.gbs 
    atoms 7    entry N          line   242 file /media/minicond

Exception: Maximum number of SCF cycles exceeded.

In [ ]:
E,_ = M.runPSI4("PBE")